In [ ]:
!pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 1.5MB 4.5MB/s 
     |████████████████████████████████| 23.9MB 1.4MB/s 
     |████████████████████████████████| 10.8MB 44.0MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.3.0 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

In [ ]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

In [ ]:
!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip

Archive:  cc.ru.300.vec.zip
  inflating: cc.ru.300.vec           
Archive:  cc.uk.300.vec.zip
  inflating: cc.uk.300.vec           


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [ ]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

Учим маппинг из одного пространства эмбеддингов в другое

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329297304153442),
 ('октябрь', 0.8311845660209656),
 ('ноябрь', 0.8278923630714417),
 ('июль', 0.8234528303146362),
 ('август', 0.8120501637458801),
 ('декабрь', 0.8039003610610962)]

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
      pred_list = ru_emb.most_similar(mapped_vectors[i:i+1])[:topn]
      pred_list = [j for j, _ in pred_list]
      num_matches += 1 if ru in pred_list else 0
    precision_val = num_matches / len(pairs)
    return precision_val

In [ ]:
print(precision([("серпень", "август")], august, topn=5))
print(precision([("серпень", "август")], august, topn=9))
print(precision([("серпень", "август")], august, topn=10))

0.0
1.0
1.0


In [ ]:
print(precision(uk_ru_test, mapping.predict(X_test)))
print(precision(uk_ru_test, mapping.predict(X_test), 5))

0.6356589147286822
0.813953488372093


Улучшаем маппинг  

In [ ]:
def learn_transform(X_train, Y_train):
    XY_dot = np.dot(X_train.T, Y_train)
    u, _, vh = np.linalg.svd(XY_dot)
    W = np.dot(u, vh)
    return W

In [ ]:
W = learn_transform(X_train, Y_train)
W.shape

(300, 300)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb['серпень'], W)])

[('апрель', 0.8237907886505127),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.8025653958320618),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001735806465149),
 ('ноябрь', 0.7934483289718628),
 ('февраль', 0.7914120554924011),
 ('июль', 0.7908109426498413),
 ('август', 0.7891016602516174),
 ('декабрь', 0.7686373591423035)]

In [ ]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6537467700258398
0.8242894056847545


Пишем переводчик

In [ ]:
with open("fairy_tale.txt", "r") as f:
    uk_sentences = [line.rstrip().lower() for line in f]

uk_sentences[:5]

['лисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .']

In [ ]:
def translate(sentence):
    preds = []
    for word in sentence.split():
      try:
        word = mapping.predict(uk_emb[word].reshape(1, -1))
        word = ru_emb.most_similar(word)[0][0]
        preds.append(word)
      except KeyError:
        preds.append(word)

    return ' '.join(preds)

In [ ]:
print(translate("."))
print(translate("1 , 3"))
print(translate("кіт побачив їжу"))

Во
от , до
кот увидел пищу


In [ ]:
for sentence in uk_sentences:
    print(f"src: {sentence}\ndst: {translate(sentence)}\n")

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – девочка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , ой и пошла раз от однії бабы добувать огнь ; лечь во избу ой и говорит из " хороший день тебе , бабуля !

src: дай мені огня " .
dst: дай мне из " Во

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вынуть со печи пирожок со маком , сладкий , ой и легла , чтобы он прохолов ; а лисичка коль и підгледала , ой притом что баба наклониться во печь , чтобы достать из , то лисичка сейчас ухватила пирожок ой и сожрать со избы , ой , бежать , весь мак со его виїла , а туда мусор на